# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'  #UPDATE THIS LINE
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [2]:
import numpy as np
import pandas as pd
np.random.seed(1234)
from rich import print
from rich.columns import Columns
from rich.panel import Panel
from rich.align import Align
from source.utils.print_util import *
from source.utils.data_preprocessing import *
import matplotlib.pyplot as plt
from fairlearn.metrics import  MetricFrame, count, false_negative_rate, false_positive_rate, selection_rate, equalized_odds_difference, demographic_parity_difference
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.reductions import ExponentiatedGradient, Moment
from fairlearn.postprocessing import ThresholdOptimizer
from aif360.datasets import BinaryLabelDataset, StructuredDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, Metric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR, OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.inprocessing import PrejudiceRemover, AdversarialDebiasing, ExponentiatedGradientReduction
from aif360.algorithms.postprocessing import RejectOptionClassification, CalibratedEqOddsPostprocessing, EqOddsPostprocessing
from aif360.algorithms import Transformer
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from tqdm.notebook import tqdm
import pickle
import time

pip install 'aif360[inFairness]'


In [3]:
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, normalize
from source.utils.config import *

In [4]:
!pip install BlackBoxAuditing

## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [5]:
begin = time.time()

#INPUT
dataset_name = "aids"

if dataset_name == "aids":
  target_variable = 'cid'
  target_variable_labels= [0,1]
  #sensible_attribute = 'race_cat'
  #sensible_attribute = 'homo_cat'
  sensible_attribute = 'age_cat'
  ignore_cols = []
  if sensible_attribute == 'race_cat':
    maps = [{1.0: 'White', 0.0: 'Non-white'}]
  elif sensible_attribute == 'homo_cat':
    maps = [{1.0: 'Non-Homo', 0.0: 'Homo'}]
  elif sensible_attribute == 'age_cat':
    maps = [{1.0: 'Adult', 0.0: 'Young'}]
  default_mappings = {
      'label_maps': [{1.0: 'Death', 0.0: 'Censored'}],
      'protected_attribute_maps': maps
  }

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

In [6]:
config = {}
config['df']= df
config['target_variable'] = target_variable
config['sensible_attribute'] = sensible_attribute
config['path_to_project'] = path_to_project
config['n_splits'] = n_splits
config['models'] = models
config['n_estimators'] = n_estimators
config['random_seed'] = random_seed

In [7]:
def unpack_config(config):
  return config['df'], config['target_variable'], config['sensible_attribute'], config['path_to_project'], config['n_splits'], config['models'], config['n_estimators'],config['random_seed']

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [8]:
df.head(5)

,pidnum,cid,time,trt,age_cat,homo_cat,race_cat,wtkg,hemo,drugs,...,gender,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820
0,260879,0,916,1,0,0,1,69.8000,0,1,...,1,0,1,0,1,1,435,459,886,855
1,150477,0,992,0,1,0,1,69.3000,0,1,...,1,1,3,0,0,1,267,272,839,842
2,50233,0,1231,2,1,0,1,72.5000,0,0,...,1,1,3,0,1,0,486,464,986,967
3,310231,0,966,3,1,1,1,73.0296,1,1,...,1,0,1,0,1,0,480,403,700,864
4,940519,1,314,0,0,1,0,56.8000,1,0,...,1,1,3,0,0,0,276,150,1279,805


## Mitigation utils


In [9]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [10]:
#print(train_test_splitting(df, n_splits))

In [11]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [12]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [13]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [14]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [15]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [16]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [17]:
#print(compute_predictions(df, target_variable, n_splits, models, n_estimators, random_seed))

In [18]:
def save_predictions_and_tests(predicted_and_test, mitigation, dataset_name, path_to_project):
  save_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, mitigation):
  mitigated_dataset.to_csv("{}/data/mitigated/mitigated-{}-{}-{}.csv".format(path_to_project,dataset_name, sensible_attribute, mitigation), sep=',', index=False, encoding='utf-8')

In [20]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

MITIGATED DATASET

ALPHA PARAMETER SET TO DEFAULT

NO MODEL ENFORCEMENT

In [21]:
def fl_cr(config):
  mitigation='fl-cr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  X_raw = df.drop(target_variable, axis=1)
  X_raw = pd.get_dummies(X_raw)
  y = df[target_variable].values

  cr = CorrelationRemover(sensitive_feature_ids=[sensible_attribute])
  X_cr = cr.fit_transform(X_raw)

  X_cr = pd.DataFrame(
      X_cr, columns = X_raw.drop(sensible_attribute, axis=1).columns
  )

  X_cr[sensible_attribute] = X_raw[sensible_attribute]
  mit_fl_cr = X_cr.copy(deep=True)
  mit_fl_cr[target_variable] = y

  save_mitigated_dataset(mit_fl_cr,path_to_project,dataset_name, mitigation)
  predictions_and_tests = compute_predictions_and_tests(mit_fl_cr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, dataset_name, mitigation,path_to_project)
  return mit_fl_cr, predictions_and_tests

In [22]:
print(fl_cr(config))

Output hidden; open in https://colab.research.google.com to view.

### Threshold Optimizer
Post-processing: change predictions, mitigated dataset is equal to the original one

SET THE MODEL TO CLASSIFIER - NO MODEL ENFORCEMENT

NO PARAMETERS

NO MITIGATED DATASET

In [23]:
def fl_to(config):
  mitigation='fl-to'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  predictions_and_tests= {}

  for model_name in tqdm(models):
    classifier =  models[model_name]
    TO = ThresholdOptimizer(estimator=classifier)
    pred_and_y = fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, TO)
    predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, path_to_project)
  return predictions_and_tests

In [24]:
print(fl_to(config))

Output hidden; open in https://colab.research.google.com to view.

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

### Configuration

In order to use [AIF360](https://aif360.mybluemix.net) we need to define a Binary Dataset and set the privileged/unprivileged groups

In [25]:
data_orig_aif = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute])

privileged_groups = [{sensible_attribute : 1}]
unprivileged_groups = [{sensible_attribute: 0}]

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

NO PARAMETER

NO MITIGATED DATASET

NO MODEL ENFORCEMENT

In [26]:
def aif360_rw(config):
  mitigation = 'aif360-rw'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    RW = Reweighing(unprivileged_groups = unprivileged_groups,
                privileged_groups = privileged_groups)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df_train.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])
      rw_dataset = RW.fit_transform(data_orig_aif)
      rw_df = rw_dataset.convert_to_dataframe()
      sample_weights= rw_df[1]["instance_weights"]


      clf.fit(X_train,Y_train,sample_weight=sample_weights)
      y_pred = clf.predict(X_test)
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

      predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, path_to_project)

  return predictions_and_tests

In [27]:
print(aif360_rw(config))

Output hidden; open in https://colab.research.google.com to view.

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

PARAMETER REPAIR LEVEL

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [28]:
def aif360_di(config):
  mitigation = 'aif360-di'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  #levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  DIR = DisparateImpactRemover(repair_level=0.5)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  train_repd = DIR.fit_transform(data_orig_aif)
  mit_aif360_di = train_repd.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_di,path_to_project,dataset_name, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_di, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, path_to_project)

  return predictions_and_tests

In [29]:
print(aif360_di(config))

Output hidden; open in https://colab.research.google.com to view.

### Learning Fair Representation

PARAMETERS k, Ax, Ay, Az maxitem e maxfun

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [30]:
def aif360_lfr(config):
  mitigation = 'aif360-lfr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  if sensible_attribute == 'homo_cat':
    kk = 147
  else:
    kk = 51

  TR = LFR(unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed= random_seed, k=kk, Ax=0.01, Ay=1.0, Az=50.0,
    verbose=1)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  TR = TR.fit(data_orig_aif, maxiter=5000, maxfun=5000)
  transf_dataset = TR.transform(data_orig_aif)
  mit_aif360_lfr = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_lfr,path_to_project,dataset_name, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_lfr, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, mitigation, path_to_project)

  return predictions_and_tests

In [31]:
results= aif360_lfr(config)
print(results)

Output hidden; open in https://colab.research.google.com to view.

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

CREATE OptTool for each dataset

PARAMETERS distortion_fun, epsilon, clist, dlist

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [32]:
def get_distortion_aids(vold,vnew):   #calculates distance between two instances of target
        bad_val=3
        OutNew=vnew["cid"]
        OutOld=vold["cid"]

        if (OutNew>OutOld):
            return bad_val
        else:
            return 0

In [33]:
'''def distortion_function(df):
  df2=df.drop(columns=['Pregnancies','DiabetesPedigreeFunction'])
  #df2 = df.copy()
  df2['BMI'] = np.where(df2['BMI'].between(18,25), 1, 0) # 1 valore giusto, 0 anomalo
  df2['Glucose'] = np.where(df2['Glucose'] < 120, 1, 0)
  df2["BloodPressure"]=np.where(df2["BloodPressure"].between(60,80),1,0)
  df2["SkinThickness"]=np.where(df2["SkinThickness"].between(22,24),1,0)
  df2["Insulin"]=np.where(df2["Insulin"].between(5,25),1,0)
  return df2'''

'def distortion_function(df):\n  df2=df.drop(columns=[\'Pregnancies\',\'DiabetesPedigreeFunction\'])\n  #df2 = df.copy()\n  df2[\'BMI\'] = np.where(df2[\'BMI\'].between(18,25), 1, 0) # 1 valore giusto, 0 anomalo\n  df2[\'Glucose\'] = np.where(df2[\'Glucose\'] < 120, 1, 0)\n  df2["BloodPressure"]=np.where(df2["BloodPressure"].between(60,80),1,0)\n  df2["SkinThickness"]=np.where(df2["SkinThickness"].between(22,24),1,0)\n  df2["Insulin"]=np.where(df2["Insulin"].between(5,25),1,0)\n  return df2'

In [34]:
import statistics

def distortion_function(df):
  list_sensible = ['race_cat', 'homo_cat', 'age_cat']
  if sensible_attribute in list_sensible:
    list_sensible.remove(sensible_attribute)

  df2=df.drop(columns=['pidnum', 'trt', 'wtkg', 'hemo', 'drugs', 'karnof','z30', 'oprior', 'preanti', 'gender', 'str2', 'treat', 'offtrt', 'cd80', 'cd820'])
  df2 = df2.drop(columns = list_sensible)
  # keeping attributes 'time', 'strat', 'symptom', 'cd40', 'cd420'    #works with 6 attributes too (z30)
  # the values used for the binarization are taken from the preprocessing: they correspond to the median of the values of the attributes
  #for each non-binary attribute, make it binary
  df2['time'] = np.where(df2['time'] < 997, 1, 0)
  df2['strat'] = np.where(df2['strat'].between(1,2), 1, 0)
  df2['cd40'] = np.where(df2['cd40'] < 340, 1, 0)
  df2['cd420'] = np.where(df2['cd420'] < 353, 1, 0)
  return df2

In [35]:
def aif360_op(config):
  mitigation = 'aif360-op'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  df_op = distortion_function(df)

  data_orig_aif_op = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df_op.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute],
    metadata = default_mappings)

  optim_options = {
    "distortion_fun": get_distortion_aids,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
  }

  predictions_and_tests = {}
  OP = OptimPreproc(OptTools, optim_options)
  dataset_orig_train, dataset_orig_test = data_orig_aif_op.split([0.7], shuffle=True)
  OP_fitted= OP.fit(dataset_orig_train)
  transf_dataset = OP_fitted.transform(data_orig_aif_op, transform_Y=True)
  mit_aif360_op = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_op,path_to_project,dataset_name, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_op, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, path_to_project)

  return predictions_and_tests

In [36]:
print(aif360_op(config))

Output hidden; open in https://colab.research.google.com to view.

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given (FIXED WITH VERSION 0.6.1 OF AIF360)

PARAMETER eta

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT



In [37]:
def aif360_pr(config):
  mitigation = 'aif360-pr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    PR = PrejudiceRemover(eta=1.0, sensitive_attr=sensible_attribute, class_attr=target_variable)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    print("TRAIN")
    print(data_orig_aif_train)
    print("TEST")
    print(data_orig_aif_test)

    PR.fit(data_orig_aif_train)
    predictions = PR.predict(data_orig_aif_test)
    mit_aif360_pr = predictions.convert_to_dataframe()[0]

    y_pred = mit_aif360_pr[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y
  print(predicted_and_real_tests)

  save_predictions_and_tests(predicted_and_real_tests, mitigation, dataset_name, path_to_project)
  return predicted_and_real_tests



In [38]:
aif360_pr(config)

TRAIN

instance weights  features                                   \
                                                       protected attribute   
                                   pidnum    time  trt             age_cat   
instance names                                                               
181                         1.0  320326.0  1010.0  0.0                 1.0   
182                         1.0  210645.0   226.0  0.0                 1.0   
183                         1.0  940534.0   265.0  2.0                 0.0   
184                         1.0  140748.0   406.0  0.0                 0.0   
185                         1.0   11131.0  1064.0  2.0                 0.0   
...                         ...       ...     ...  ...                 ...   
1813                        1.0  181296.0  1120.0  2.0                 0.0   
1814                        1.0  251032.0  1111.0  3.0                 0.0   
1815                        1.0  620018.0   993.0  1.0                 1.0   
1816                        1.0  120295.0  1160.0  2.0                 1.0   
1817                        1.0  150431.0   770.0  1.0                 0.0   

                                                       ...                     \
                                                       ...                      
               homo_cat race_cat      wtkg hemo drugs  ... str2 strat symptom   
instance names                                         ...                      
181                 0.0      1.0  83.28096  0.0   0.0  ...  0.0   1.0     0.0   
182                 1.0      1.0  66.67920  0.0   0.0  ...  1.0   3.0     1.0   
183                 1.0      1.0  62.86896  1.0   0.0  ...  1.0   2.0     0.0   
184                 0.0      1.0  71.21520  0.0   0.0  ...  1.0   3.0     1.0   
185                 0.0      1.0  75.50000  0.0   0.0  ...  0.0   1.0     0.0   
...                 ...      ...       ...  ...   ...  ...  ...   ...     ...   
1813                0.0      1.0  62.50000  0.0   0.0  ...  1.0   2.0     0.0   
1814                0.0      1.0  72.12240  0.0   0.0  ...  1.0   3.0     0.0   
1815                1.0      0.0  55.50000  0.0   0.0  ...  0.0   1.0     1.0   
1816                0.0      1.0  81.64800  0.0   0.0  ...  1.0   3.0     0.0   
1817                0.0      1.0  87.00000  0.0   0.0  ...  1.0   3.0     1.0   

                                                          labels  
                                                                  
               treat offtrt   cd40  cd420    cd80   cd820         
instance names                                                    
181              0.0    0.0  420.0  465.0   700.0   675.0    0.0  
182              0.0    1.0  278.0  224.0   955.0   964.0    1.0  
183              1.0    1.0  445.0  286.0   810.0   678.0    0.0  
184              0.0    1.0  320.0  199.0  1359.0  1300.0    1.0  
185              1.0    1.0  400.0  480.0  1250.0  1420.0    0.0  
...              ...    ...    ...    ...     ...     ...    ...  
1813             1.0    1.0  226.0  313.0   950.0   816.0    0.0  
1814             1.0    0.0  259.0  311.0  1238.0   901.0    0.0  
1815             1.0    1.0  300.0  240.0   790.0   520.0    0.0  
1816             1.0    0.0  211.0  247.0  1289.0  1073.0    0.0  
1817             1.0    0.0  317.0  475.0   903.0  1226.0    1.0  

[1637 rows x 25 columns]

TEST

instance weights  features                                   \
                                                       protected attribute   
                                   pidnum    time  trt             age_cat   
instance names                                                               
0                           1.0  260879.0   916.0  1.0                 0.0   
1                           1.0  150477.0   992.0  0.0                 1.0   
2                           1.0   50233.0  1231.0  2.0                 1.0   
3                           1.0  310231.0   966.0  3.0                 1.0   
4                           1.0  940519.0   314.0  0.0                 0.0   
...                         ...       ...     ...  ...                 ...   
176                         1.0   30090.0  1090.0  2.0                 0.0   
177                         1.0  270852.0  1108.0  2.0                 1.0   
178                         1.0  250225.0  1143.0  0.0                 1.0   
179                         1.0  620058.0   423.0  1.0                 0.0   
180                         1.0   10915.0  1145.0  0.0                 0.0   

                                                      ...                     \
                                                      ...                      
               homo_cat race_cat     wtkg hemo drugs  ... str2 strat symptom   
instance names                                        ...                      
0                   0.0      1.0  69.8000  0.0   1.0  ...  0.0   1.0     0.0   
1                   0.0      1.0  69.3000  0.0   1.0  ...  1.0   3.0     0.0   
2                   0.0      1.0  72.5000  0.0   0.0  ...  1.0   3.0     0.0   
3                   1.0      1.0  73.0296  1.0   1.0  ...  0.0   1.0     0.0   
4                   1.0      0.0  56.8000  1.0   0.0  ...  1.0   3.0     0.0   
...                 ...      ...      ...  ...   ...  ...  ...   ...     ...   
176                 0.0      1.0  71.4000  0.0   0.0  ...  1.0   3.0     0.0   
177                 0.0      1.0  73.9368  0.0   0.0  ...  1.0   2.0     0.0   
178                 0.0      1.0  72.1224  0.0   0.0  ...  0.0   1.0     0.0   
179                 0.0      0.0  80.5000  0.0   0.0  ...  0.0   1.0     0.0   
180                 0.0      1.0  75.7000  0.0   0.0  ...  1.0   3.0     0.0   

                                                          labels  
                                                                  
               treat offtrt   cd40  cd420    cd80   cd820         
instance names                                                    
0                1.0    1.0  435.0  459.0   886.0   855.0    0.0  
1                0.0    1.0  267.0  272.0   839.0   842.0    0.0  
2                1.0    0.0  486.0  464.0   986.0   967.0    0.0  
3                1.0    0.0  480.0  403.0   700.0   864.0    0.0  
4                0.0    0.0  276.0  150.0  1279.0   805.0    1.0  
...              ...    ...    ...    ...     ...     ...    ...  
176              1.0    0.0  352.0  417.0   983.0  1157.0    0.0  
177              1.0    0.0  347.0  333.0   891.0   594.0    0.0  
178              0.0    0.0  380.0  580.0   520.0   770.0    0.0  
179              1.0    0.0  170.0  160.0   570.0   350.0    1.0  
180              0.0    0.0  416.0  426.0   991.0  1017.0    0.0  

[181 rows x 25 columns]

TRAIN

...

instance weights  features                                   \
                                                       protected attribute   
                                   pidnum    time  trt             age_cat   
instance names                                                               
0                           1.0  260879.0   916.0  1.0                 0.0   
1                           1.0  150477.0   992.0  0.0                 1.0   
2                           1.0   50233.0  1231.0  2.0                 1.0   
3                           1.0  310231.0   966.0  3.0                 1.0   
4                           1.0  940519.0   314.0  0.0                 0.0   
...                         ...       ...     ...  ...                 ...   
1624                        1.0  230073.0  1088.0  3.0                 1.0   
1625                        1.0   11970.0  1073.0  3.0                 0.0   
1626                        1.0  160370.0  1065.0  2.0                 0.0   
1627                        1.0  270739.0   350.0  2.0                 0.0   
1628                        1.0  220438.0  1091.0  1.0                 1.0   

                                                       ...                     \
                                                       ...                      
               homo_cat race_cat      wtkg hemo drugs  ... str2 strat symptom   
instance names                                         ...                      
0                   0.0      1.0   69.8000  0.0   1.0  ...  0.0   1.0     0.0   
1                   0.0      1.0   69.3000  0.0   1.0  ...  1.0   3.0     0.0   
2                   0.0      1.0   72.5000  0.0   0.0  ...  1.0   3.0     0.0   
3                   1.0      1.0   73.0296  1.0   1.0  ...  0.0   1.0     0.0   
4                   1.0      0.0   56.8000  1.0   0.0  ...  1.0   3.0     0.0   
...                 ...      ...       ...  ...   ...  ...  ...   ...     ...   
1624                0.0      1.0   61.2360  0.0   0.0  ...  1.0   3.0     0.0   
1625                0.0      1.0   91.6000  0.0   0.0  ...  1.0   2.0     0.0   
1626                0.0      1.0   79.9000  0.0   0.0  ...  1.0   2.0     0.0   
1627                1.0      1.0  101.0000  0.0   0.0  ...  1.0   2.0     0.0   
1628                1.0      0.0   58.0000  0.0   0.0  ...  0.0   1.0     0.0   

                                                          labels  
                                                                  
               treat offtrt   cd40  cd420    cd80   cd820         
instance names                                                    
0                1.0    1.0  435.0  459.0   886.0   855.0    0.0  
1                0.0    1.0  267.0  272.0   839.0   842.0    0.0  
2                1.0    0.0  486.0  464.0   986.0   967.0    0.0  
3                1.0    0.0  480.0  403.0   700.0   864.0    0.0  
4                0.0    0.0  276.0  150.0  1279.0   805.0    1.0  
...              ...    ...    ...    ...     ...     ...    ...  
1624             1.0    0.0  286.0  325.0   602.0   568.0    0.0  
1625             1.0    1.0  229.0  288.0   866.0   996.0    0.0  
1626             1.0    0.0  468.0  406.0  1357.0   797.0    0.0  
1627             1.0    1.0  337.0  378.0   702.0   643.0    0.0  
1628             1.0    0.0  413.0  623.0  1106.0  1102.0    0.0  

[1629 rows x 25 columns]

TEST

instance weights  features                                   \
                                                       protected attribute   
                                   pidnum    time  trt             age_cat   
instance names                                                               
1629                        1.0   60932.0   583.0  3.0                 1.0   
1630                        1.0   20703.0  1027.0  2.0                 1.0   
1631                        1.0  270042.0  1151.0  0.0                 0.0   
1632                        1.0   81444.0  1083.0  3.0                 1.0   
1633                        1.0  320198.0   835.0  2.0                 0.0   
...                         ...       ...     ...  ...                 ...   
1805                        1.0  140400.0  1121.0  1.0                 1.0   
1806                        1.0  960017.0   717.0  3.0                 1.0   
1807                        1.0   50653.0   971.0  3.0                 0.0   
1808                        1.0  940531.0  1115.0  1.0                 0.0   
1809                        1.0  230286.0  1090.0  1.0                 0.0   

                                                       ...                     \
                                                       ...                      
               homo_cat race_cat      wtkg hemo drugs  ... str2 strat symptom   
instance names                                         ...                      
1629                0.0      1.0  89.35920  0.0   0.0  ...  0.0   1.0     0.0   
1630                0.0      0.0  78.92640  0.0   1.0  ...  0.0   1.0     0.0   
1631                0.0      0.0  78.01920  0.0   0.0  ...  1.0   3.0     0.0   
1632                0.0      1.0  73.80000  0.0   0.0  ...  0.0   1.0     1.0   
1633                0.0      1.0  59.87520  0.0   0.0  ...  0.0   1.0     1.0   
...                 ...      ...       ...  ...   ...  ...  ...   ...     ...   
1805                0.0      1.0  85.50000  0.0   0.0  ...  1.0   3.0     0.0   
1806                1.0      1.0  76.65840  1.0   0.0  ...  1.0   3.0     0.0   
1807                0.0      0.0  92.53440  0.0   0.0  ...  0.0   1.0     0.0   
1808                1.0      1.0  62.86896  1.0   0.0  ...  1.0   2.0     0.0   
1809                1.0      1.0  68.49360  0.0   0.0  ...  1.0   3.0     1.0   

                                                          labels  
                                                                  
               treat offtrt   cd40  cd420    cd80   cd820         
instance names                                                    
1629             1.0    1.0  225.0  351.0   225.0   304.0    0.0  
1630             1.0    0.0  304.0  436.0   486.0   515.0    0.0  
1631             0.0    0.0  374.0  353.0   671.0   742.0    0.0  
1632             1.0    0.0  360.0  391.0  1180.0  1020.0    0.0  
1633             1.0    1.0  170.0  288.0   348.0   376.0    0.0  
...              ...    ...    ...    ...     ...     ...    ...  
1805             1.0    1.0  522.0  408.0  1682.0  1417.0    0.0  
1806             1.0    1.0  265.0  248.0   967.0  1080.0    0.0  
1807             1.0    0.0  617.0  710.0   935.0  1009.0    0.0  
1808             1.0    0.0  229.0  139.0  1041.0   662.0    0.0  
1809             1.0    0.0  347.0  342.0   732.0   550.0    0.0  

[181 rows x 25 columns]

{
    0: {
        'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0]),
        'y_pred': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0]),
        's_test': array([0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0])
    },
    1: {
        'y_test': array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0]),
        'y_pred': array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
 

{0: {'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
         0, 0, 0, 1, 0]),
  'y_pred': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 

### Exponentiated Gradient Reduction
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETERS constraint e estimator

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [39]:
def aif360_er(config):
  mitigation = 'aif360-er'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  predictions_and_tests = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    #lmod = DecisionTreeClassifier(max_depth=None)
    lmod = LogisticRegression(solver='lbfgs', max_iter=1000)

    exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints='DemographicParity', drop_prot_attr=False)
    exp_grad_red.fit(data_orig_aif_train)
    predictions = exp_grad_red.predict(data_orig_aif_test)

    mit_aif360_er = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_er[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

  predictions_and_tests = pred_and_y

  print(predictions_and_tests)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, path_to_project)
  return predictions_and_tests


In [40]:
aif360_er(config)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:185: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updatin

{
    0: {
        'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0]),
        'y_pred': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0]),
        's_test': array([0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0])
    },
    1: {
        'y_test': array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0]),
        'y_pred': array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
 

{0: {'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
         0, 0, 0, 1, 0]),
  'y_pred': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 

### Adversarial Debiasing
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETER SESSION

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [41]:
def aif360_ad(config):
  mitigation = 'aif360-ad'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):
    # Create a new session for each fold
    sess = tf.Session()
    # Added a unique scope name using the fold index
    AD = AdversarialDebiasing(privileged_groups = privileged_groups,
                        unprivileged_groups = unprivileged_groups,
                        scope_name=f'plain_classifier_{i}', # Changed scope name
                        debias=False,
                        sess=sess)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    AD.fit(data_orig_aif_train)

    predictions = AD.predict(data_orig_aif_test)

    mit_aif360_ad = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_ad[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y

    # Close the session after each fold
    sess.close()
  print(predicted_and_real_tests)
  save_predictions_and_tests(predicted_and_real_tests, mitigation, dataset_name, path_to_project)
  return predicted_and_real_tests

In [42]:
aif360_ad(config)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 2624.658691
epoch 1; iter: 0; batch classifier loss: 2997.114258
epoch 2; iter: 0; batch classifier loss: 3704.157227
epoch 3; iter: 0; batch classifier loss: 2557.924561
epoch 4; iter: 0; batch classifier loss: 1822.324097
epoch 5; iter: 0; batch classifier loss: 1945.459473
epoch 6; iter: 0; batch classifier loss: 1607.862549
epoch 7; iter: 0; batch classifier loss: 1948.829102
epoch 8; iter: 0; batch classifier loss: 2437.086426
epoch 9; iter: 0; batch classifier loss: 1454.619141
epoch 10; iter: 0; batch classifier loss: 1145.590576
epoch 11; iter: 0; batch classifier loss: 1683.339844
epoch 12; iter: 0; batch classifier loss: 1298.667236
epoch 13; iter: 0; batch classifier loss: 1257.223145
epoch 14; iter: 0; batch classifier loss: 1227.164673
epoch 15; iter: 0; batch classifier loss: 1283.570923
epoch 16; iter: 0; batch classifier loss: 1152.561890
epoch 17; iter: 0; batch classifier loss: 1298.889160
epoch 18; iter: 0; batch classifier lo

{
    0: {
        'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0]),
        's_test': array([0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0])
    },
    1: {
        'y_test': array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        's_test': array([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
 

{0: {'y_test': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
         0, 0, 0, 1, 0]),
  'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Post-processing

In the first phase there is the model building: the same for Calibrated Equalized Post-processing, Equalize Odds Post_processing and Reject Option Classification


In [43]:
def compute_predictions(x_train, y_train, x_test, model):
  model.fit(x_train, y_train)
  return model.predict(x_test)

### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

PARAMETER cost_constraint

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [44]:
def aif360_ce(config):
  mitigation = 'aif360-ce'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    CEPP = CalibratedEqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, cost_constraint='weighted', seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      # data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_train.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_test.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      # cpp = CEPP.fit(data_orig_valid, data_orig_valid_pred)
      # predictions = cpp.predict(data_orig_aif_test)

      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred


      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])


      # Fit the post-processing technique with these two datasets i.e. test, pred
      predictions = CEPP.fit_predict(data_orig_aif_test, data_orig_aif_pred)
      mit_aif360_cpp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_cpp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, mitigation, path_to_project)

  return predictions_and_tests

In [45]:
aif360_ce(config)

Output hidden; open in https://colab.research.google.com to view.

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one (Fixed updating AIF360 Version)

NO PARAMETER

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [46]:
def aif360_eo(config):
  mitigation = 'aif360-eo'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      # data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_train.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_test.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      # eopp = EOPP.fit(data_orig_valid, data_orig_valid_pred)
      # predictions = eopp.predict(data_orig_aif_test)
      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred


      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])


      # Fit the post-processing technique with these two datasets i.e. test, pred
      predictions = EOPP.fit_predict(data_orig_aif_test, data_orig_aif_pred)

      mit_aif360_eopp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_eopp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, mitigation, path_to_project)

  return predictions_and_tests

In [47]:
aif360_eo(config)

Output hidden; open in https://colab.research.google.com to view.

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO MITIGATED DATASET

PARAMETERS: low_class_thresh,high_class_thresh, num_class_thresh, num_ROC_margin, metric_name, metric_ub, metric_lb

NO EMBEDDED SCORE FUCTION

NO MODEL ENFORCEMENT

In [48]:
def aif360_roc(config):
  mitigation = 'aif360-roc'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name="Statistical parity difference", metric_ub=0.05, metric_lb=-0.05)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values.astype(int)

      # data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_train.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      # unfavorable_label = 0,
      # df = df_test.copy(),
      # label_names = [target_variable],
      # protected_attribute_names = [sensible_attribute])

      # data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      # roc = ROC.fit(data_orig_valid, data_orig_valid_pred)
      # predictions = roc.predict(data_orig_aif_test)
      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred

      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      # Fit the post-processing technique with these two datasets i.e. test, pred
      predictions = ROC.fit_predict(data_orig_aif_test, data_orig_aif_pred)
      mit_aif360_roc = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_roc[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, mitigation, path_to_project)

  return predictions_and_tests

In [49]:
aif360_roc(config)

Output hidden; open in https://colab.research.google.com to view.

# Measure time to run

In [50]:
end = time.time()
print(f"Total runtime of the program is {end - begin}")

Total runtime of the program is 695.3439514636993